In [ ]:
#import libraries

import statsmodels.api as sm
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from statsmodels.stats.anova import anova_lm

In [ ]:
#load crime data from data folder

df = pd.read_csv('../data/raw/crime.csv')

df.head()

,state,murder.rate,poverty,high.school,college,single.parent,unemployed,metropolitan,region
0,Alabama,7.4,14.7,77.5,20.4,26.0,4.6,70.2,South
1,Alaska,4.3,8.4,90.4,28.1,23.2,6.6,41.6,West
2,Arizona,7.0,13.5,85.1,24.6,23.5,3.9,87.9,West
3,Arkansas,6.3,15.8,81.7,18.4,24.7,4.4,49.0,South
4,California,6.1,14.0,81.2,27.5,21.8,4.9,96.7,West


In [ ]:
# declare variables (drop murder rate from X)

X = df.drop(columns=['murder.rate', 'state', 'region'])
y = df['murder.rate']

In [16]:
# split the data in preparation of running a Linear Regression model

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1242)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(40, 6) (10, 6) (40,) (10,)


In [ ]:
# run a multiple linear regression model to predict the murder rate

model = LinearRegression()
model.fit(X_train, y_train)

print (model.coef_)
print(model.intercept_)

[ 0.00790884 -0.09705277 -0.01548963  0.45389937  0.3391889   0.03907391]
-0.9639947020071737


In [ ]:
# run anova

model_anova = sm.OLS(y_train, X_train).fit()
print(model_anova.summary())

                                 OLS Regression Results                                
Dep. Variable:            murder.rate   R-squared (uncentered):                   0.919
Model:                            OLS   Adj. R-squared (uncentered):              0.904
Method:                 Least Squares   F-statistic:                              63.96
Date:                Tue, 21 Oct 2025   Prob (F-statistic):                    4.39e-17
Time:                        14:08:19   Log-Likelihood:                         -75.158
No. Observations:                  40   AIC:                                      162.3
Df Residuals:                      34   BIC:                                      172.4
Df Model:                           6                                                  
Covariance Type:            nonrobust                                                  
                    coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------

In [ ]:
# get rid of the not statistically significant variables

X_reduced = X.drop(columns=['poverty','college','unemployed'])
model_reduced = sm.OLS(y_train, X_train).fit()